# IMPORT LIBRARIES AND VARIABLES

In [1]:
import pandas as pd
import os

In [2]:
traits_list =  ['SBP','DBP','FNBD',  'DEP_ukbb',  'AUT_ukbb','LDL','BMI',  'GLC', 'TC','TG','T2D',
              'PBC', 'T1D', 'RA','SLE', 'MS',  'CD', 'CEL',
              'BC_gwas', 'LeuC_gwas', 'CD4regTC_gwas','nCD8_gwas', 
              'HL_ukbb', 'LEU_ukbb','NHL_ukbb','NHL_gwas','PLHMN_ukbb', 'LL_C91_ukbb','CLL_ukbb','LL_gwas', 'LL_ukbb', 
              'MC_gwas','MWCC_gwas','MCHC','MCH',  'MCV',  'MPC',   'PC',   'RBCC']

traits_order = traits_list

## traits classification
traits_class = {'other':['SBP','DBP','FNBD',  'DEP_ukbb',  'AUT_ukbb','LDL','BMI','GLC','TC','TG','T2D',],
               'autoimmune':['PBC', 'T1D', 'RA','SLE', 'MS',  'CD', 'CEL'],
               'lymphoid':['BC_gwas', 'LeuC_gwas', 'CD4regTC_gwas','nCD8_gwas' ],
               'cancer':['HL_ukbb', 'LEU_ukbb','NHL_ukbb','NHL_gwas','PLHMN_ukbb', 'LL_C91_ukbb','CLL_ukbb',
                        'LL_gwas', 'LL_ukbb'],
               'myeloid':[ 'MC_gwas','MWCC_gwas','MCHC','MCH','MCV','MPC','PC','RBCC']
              }

# DATASETS

In [3]:
path_bloodprecursors_files = "/home/plopez/scratch/method/COGS/results/repeat_1cM_merge4GWAS_precursors_bloodprecursors//"
outpath = "/home/plopez/scratch/method/COGS/6.1.Pre_Reactome"

In [4]:
#blood + precursors
blpre_dict = {}
for filename in os.listdir(path_bloodprecursors_files):
    if filename.startswith('SNPs_tables'):
        continue
    trait = filename.split("_geneScores")[0]
    blpre = pd.read_csv(os.path.join(path_bloodprecursors_files,filename),sep="\t")
    blpre_dict[trait] = blpre

# EXAMPLE: BLOOD + PRECURSORS DATASET - TABLES OF PRIORITIZED GENES (ONE TABLE PER TRAIT)

## BLOOD+PRECURSORS: TABLE ALL GENES ALL TRAITS

In [6]:
trait = traits_order[0]
# for trait in traits_order:
conserve = blpre_dict[trait]
# conserve = conserve[['ensg','name','biotype','all_gene_score']]
conserve.index = conserve.ensg
# conserve.ensg = conserve.index
conserve = conserve[['name','biotype','all_gene_score']]
conserve = conserve.rename(columns = {'all_gene_score':trait})


for i in range(1,len(traits_order)):
    new2 = blpre_dict[traits_order[i]][['ensg','all_gene_score']]
    new2.index = new2.ensg
    new2 = new2.drop(labels=['ensg'],axis=1)
    new2 = new2.rename(columns = {'all_gene_score':traits_order[i]})
    
    new = blpre_dict[traits_order[i]][['ensg','all_gene_score','name','biotype']]
    new.index = new.ensg
    new = new.drop(labels=['ensg'],axis=1)
    new = new.rename(columns = {'all_gene_score':traits_order[i]})
    
    conserve = pd.concat([conserve,new2], axis = 1, join='outer')
    names = conserve[['name','biotype']].fillna(value=new[['name','biotype']])
    conserve = conserve.drop(labels=['name','biotype'],axis=1)
    conserve = pd.concat([names,conserve], axis = 1, join='outer')
    
#conserve.to_csv(os.path.join(outpath,"GeneScore_allTraits_allGenes_bloodprecursors.tsv"),sep="\t")

### Select only prioritized genes

In [8]:
l = ['name', 'biotype', 'max_value', 'trait_highest_score']
l.extend(traits_list)

In [9]:
conserve_values = conserve.drop(labels=['name','biotype'],axis=1)
conserve_values['max_value'] = conserve_values.max(axis=1,skipna=True)
conserve_values = conserve_values[conserve_values['max_value'] >= 0.5]
conserve_values['trait_highest_score'] = conserve_values.idxmax(axis=1)

conserve_names = conserve[['name','biotype']]

final = pd.concat([conserve_names,conserve_values], axis = 1, join='inner')
final = final.sort_values(by=['max_value'],ascending=False)
cols = ['name', 'biotype', 'max_value', 'trait_highest_score']
cols.extend(traits_list)
final = final[cols]

#final.to_csv(os.path.join(outpath,"GeneScore_allTraits_prioritizedGenes_bloodprecursors.tsv"),sep="\t")

In [18]:
final

,name,biotype,max_value,trait_highest_score,SBP,DBP,FNBD,DEP_ukbb,AUT_ukbb,LDL,...,LL_gwas,LL_ukbb,MC_gwas,MWCC_gwas,MCHC,MCH,MCV,MPC,PC,RBCC
ensg,,,,,,,,,,,,,,,,,,,,,
ENSG00000162618,ADGRL4,protein_coding,10.000117,MWCC_gwas,-,-,-,-,-,-,...,-,-,-,++,++,-,-,-,-,-
ENSG00000101076,HNF4A,protein_coding,6.999323,TC,-,-,-,-,-,-,...,-,-,+,++,-,-,-,-,+,-
ENSG00000197249,SERPINA1,protein_coding,6.568248,PC,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,++,-
ENSG00000144648,ACKR2,protein_coding,5.999338,MC_gwas,-,-,-,-,-,-,...,-,-,++,++,-,-,-,-,-,-
ENSG00000021826,CPS1,protein_coding,5.306767,MCV,-,-,-,-,-,-,...,-,-,-,+,-,++,++,++,++,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000243943,ZNF512,protein_coding,0.500000,TG,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
ENSG00000115216,NRBP1,protein_coding,0.500000,TG,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
ENSG00000119777,TMEM214,protein_coding,0.500000,TG,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


### Convert to +/++/-

In [11]:
conserve_values = conserve.drop(labels=['name','biotype'],axis=1)
conserve_values['max_value'] = conserve_values.max(axis=1,skipna=True)
conserve_values = conserve_values[conserve_values['max_value'] >= 0.5]

conserve_values2 = conserve.drop(labels=['name','biotype'],axis=1)
conserve_values2['max_value'] = conserve_values2.max(axis=1,skipna=True)
conserve_values2 = conserve_values2[conserve_values2['max_value'] >= 0.5]
conserve_values2['trait_highest_score'] = conserve_values2.idxmax(axis=1)

conserve_values2 = conserve_values2[['max_value','trait_highest_score']]

conserve_names = conserve[['name','biotype']]

conserve_values = conserve_values.apply(lambda x: ['++' if y >= 0.9 else '+' if y >= 0.5 else '-' for y in x])
conserve_values = conserve_values.drop(labels=['max_value'],axis=1)

final = pd.concat([conserve_names,conserve_values,conserve_values2], axis = 1, join='inner')


final = final.sort_values(by=['max_value'],ascending=False)
final = final[cols]

#final.to_csv(os.path.join(outpath,"GeneScore_allTraits_prioritizedGenes_boolean_bloodprecursors.tsv"),sep="\t")

## Generate Table genes Id prioritized by trait

In [13]:
dataset = "bloodprecursors"

for classname, classlist in traits_class.items():
    print(classname,classlist)
                                                
    i = 0

    trait = classlist[i]

    # print(trait)
    tlist = conserve[conserve[trait] >= 0.5][['name',trait]]
    tlist = tlist.sort_values(by=trait,axis=0, ascending=False)
    tlist = tlist.drop(columns=trait)
    tlist = tlist.rename(columns={'name':trait})
    tlist = tlist.reset_index()
    tlist = tlist.drop(columns='ensg')

    tlist_total = tlist

    for i in range(1,len(classlist)):
        trait = classlist[i]
    #     print(trait)
        tlist = conserve[conserve[trait] >= 0.5][['name',trait]]
        tlist = tlist.sort_values(by=trait,axis=0, ascending=False)
        tlist = tlist.drop(columns=trait)
        tlist = tlist.rename(columns={'name':trait})
        tlist = tlist.reset_index()
        tlist = tlist.drop(columns='ensg')

        tlist_total = pd.concat([tlist_total,tlist],axis=1)


    #tlist_total.to_csv(os.path.join(outpath,"{}_list".format(dataset),"List_prioritizedGenes_{}_{}.tsv".format(dataset,classname)),sep="\t",index=False)

other ['SBP', 'DBP', 'FNBD', 'DEP_ukbb', 'AUT_ukbb', 'LDL', 'BMI', 'GLC', 'TC', 'TG', 'T2D']
autoimmune ['PBC', 'T1D', 'RA', 'SLE', 'MS', 'CD', 'CEL']
lymphoid ['BC_gwas', 'LeuC_gwas', 'CD4regTC_gwas', 'nCD8_gwas']
cancer ['HL_ukbb', 'LEU_ukbb', 'NHL_ukbb', 'NHL_gwas', 'PLHMN_ukbb', 'LL_C91_ukbb', 'CLL_ukbb', 'LL_gwas', 'LL_ukbb']
myeloid ['MC_gwas', 'MWCC_gwas', 'MCHC', 'MCH', 'MCV', 'MPC', 'PC', 'RBCC']
